# IMPORT

In [5]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.oauth2 as oauth2
import json
import pandas as pd
import numpy as np
import requests
import re
import os
import datetime
from datetime import date, timedelta
from datetime import datetime
import glob
import time 

In [6]:
pd.set_option('display.max_columns', None)

# AUTHENTIFICATION
* guide for authentification with Spotify API: https://developer.spotify.com/documentation/general/guides/authorization-guide/
* calling personal credentials from environment https://able.bio/rhett/how-to-set-and-get-environment-variables-in-python--274rgt5
* open in terminal: touch ~/.bash_profile; open ~/.bash_profile
* implement: source ~/.bash_profile
* get new token https://developer.spotify.com/console/put-playlist/


#### Create and refresh token
run this to  refresh the token. Token runs out every 3600 seconds.

In [11]:
my_client_id=os.getenv('spoti_CLIENT_ID')
my_client_secret=os.getenv('spoti_CLIENT_SECRET')

my_client_id=os.getenv('spoti_CLIENT_ID')
my_client_secret=os.getenv('spoti_CLIENT_SECRET')
sp=spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=my_client_id, client_secret=my_client_secret))


#token to pull songs of an individual usersp_pers=spotipy.Spotify(cache_token_personal)
cache_token_personal= '' 
sp1=spotipy.Spotify(cache_token_personal)


# CRAWLING FOR CHARTS
HERE WE DOWNLOAD CSVs FORM www.spotifycharts.com

### Function
downloads CSVs from spotifycharts.com given the countries of choice and the time interval set below

* Note: this depends on the availability of the CSVs on spoitifycharts.com . Error may occure if we are trying to pull data for a day spotify does not provide charts data for. Slightly adapting frequency or start and end date might shift you around thos "data holes"

In [5]:
#DOWNLOAD LOOP FOR COUNTRY AND FOR TIME PERIOD
# prepare a folder named 'data' with two subfolders named 'viral' and 'regional'
def bulk_charts_download(country_charts, date_list):
    for country in country_charts:
        for date in date_list:
            url= f'https://spotifycharts.com/{what_charts}/{country}/daily/{date}/download'
            r = requests.get(url, allow_redirects=True)
            open(f'data/{what_charts}/{date}-{country}-{what_charts}.csv', 'wb').write(r.content)

### SET PARAMETERS FOR CRAWLING

#### Set Start and End Date

In [6]:
startdate = date(2018, 1, 1)   # start date
enddate = date(2021,1,30)   # end date
delta = enddate - startdate       # as timedelta

date_list=[]

for i in range(delta.days+1):
    day = startdate + timedelta(days=i)
    date_list.append(day.strftime("%Y-%m-%d"))

#### Chose frequency
between start date and end date how often do we pull charts data

In [7]:
date_list=date_list[0::30] #every 30 days

#### Country Codes

In [8]:
#enter list of country codes
country_charts=['us','br','jp','de','ca','gb','tr','is','mx','za','au','ar','eg','es','it','ma','nz','id','ru','ua','in','']
#HELP: LIST OF COUNTRY CHARTS
print(sp.country_codes)

['AD', 'AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'EC', 'SV', 'EE', 'FI', 'FR', 'DE', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'ID', 'IE', 'IT', 'JP', 'LV', 'LI', 'LT', 'LU', 'MY', 'MT', 'MX', 'MC', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'ES', 'SK', 'SE', 'CH', 'TW', 'TR', 'GB', 'US', 'UY']


#### Which charts are we crawling
Options: viral charts or regional top200

In [9]:
# VIRAL CHART sample URL https://spotifycharts.com/viral/us/daily/2020-03-13/download
# thats how i want filename to be safed regional-us-weekly-2020-03-13--2020-03-20.csv
# enter as string 'viral' for viral top 50 or 'regional' for local top 200
what_charts='viral'

#### Trigger download CSVs to folder: Calls function with parameters set above

In [10]:
bulk_charts_download(country_charts, date_list)

# CREATE DATAFRAMES

#### create dataframe from csvs
* intermediate step before adding music features

#### FUNCTION CSVs to Data Frames
* additionally extracts data on country, date and song IDs form the original URLs


In [12]:
def csv_to_dataframe(what_charts):
    path = f'data/{what_charts}/'                     # use your path
    all_files = glob.glob(os.path.join(path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent

    df = [] # pd.concat takes a list of dataframes as an agrument
    for csv in all_files:
        
        # if condition since csv of top200 look differently than csvs from viral charts.
        # top 200 sharts do have a disclaimer not in the first line that messes with our columns upon import
        # we need to igonre it
        if what_charts=='regional': 
            frame = pd.read_csv(csv,skiprows=[0],error_bad_lines=False, warn_bad_lines=True)
            frame=frame.drop('Streams',axis=1)
        #this is the read in for viral charts csvs
        else:
            frame = pd.read_csv(csv,skiprows=[],error_bad_lines=False, warn_bad_lines=True)
        
        frame['filename'] = os.path.basename(csv)
        df.append(frame)
        
    #concat list of dataframes into 1
    df=pd.concat(df, ignore_index=True) 
    
    #finetuning columns with regex
    df['ID']= re.findall(r'\d\w+',str(list(df['URL']))) #extract ID from URL
    df['date']=re.findall(r'\d\d\d\d-\d\d-\d\d', str(list(df['filename'])))
    df['date']=pd.to_datetime(df.date, format="%Y-%m-%d", yearfirst=True)
    df['country']=re.findall(r'-[a-z][a-z]-', str(list(df['filename'])))
    df['country']=re.findall(r'[a-z][a-z]',str(list(df['country'])))
    
    # indicates if song is from viral charts or top 200 charts/ 'regional' indicates its part of  regional top200 charts hence we label it 'top200'
    df['group']=np.where(what_charts=='regional','top200',what_charts) 
    return df

In [13]:
viral_df_basic=csv_to_dataframe('viral')#pass in bracket if 'viral' or 'regional'
len(viral_df_basic)

34250

# SUBSETTING VIRAL CHARTS
* to make the project feasible for my processing means I need to subset
* my machine is able to make approx 6300 calls to the api per hour
* subsetting to the viral top 20 of 22 countries between 01/01/2017 and 31/12/2020 at 12 sample dates
* ending up with almost 5872 ids to represent 13827 entries

In [14]:
# unique song id: plenty of songs are in teh charts multiple time for different countries and at different dates
# lets see how many IDs are unique since we dont want to pull features for the same song multiple times
unique_ids=list(viral_df_basic.ID.unique())
len(unique_ids)

14108

In [15]:
# lets return all unique IDs of songs that are at one point place in the top20 viral charts in any country at any time

unique_ids_of_top20viral=set(viral_df_basic['ID'][viral_df_basic['ID'].isin (unique_ids) & viral_df_basic['Position'].isin(list(range(1,21)))])
len(unique_ids_of_top20viral)

6008

In [16]:
# using the top 20 of each countries viral charts at all sample points results in 5872 unique IDs
# thats below my 6300 ongs threashhold

In [17]:
#
viral_df_basic_top20=viral_df_basic[viral_df_basic['Position'].isin(list(range(1,21)))]
viral_df_basic_top20.shape 

(13700, 9)

In [18]:
len(viral_df_basic_top20.ID.unique())

6008

# INSPECT DATAFRAME

In [19]:
viral_df_basic_top20.head()

Position        Track Name          Artist  \
0         1            Sucker  Jonas Brothers   
1         2           Undrunk        FLETCHER   
2         3  Don't Call Me Up           Mabel   
3         4         Moonlight         Gaullin   
4         5   i'm so tired...            Lauv   

                                                 URL                 filename  \
0  https://open.spotify.com/track/4y3OI86AEP6PQoD...  2019-03-27-in-viral.csv   
1  https://open.spotify.com/track/5SHhPFh68OhUmuR...  2019-03-27-in-viral.csv   
2  https://open.spotify.com/track/5WHTFyqSii0lmT9...  2019-03-27-in-viral.csv   
3  https://open.spotify.com/track/0A5gdlrpAuQqZ2i...  2019-03-27-in-viral.csv   
4  https://open.spotify.com/track/7COXchtUOMd6uIT...  2019-03-27-in-viral.csv   

                       ID       date country  group  
0  4y3OI86AEP6PQoDE6olYhO 2019-03-27      in  viral  
1  5SHhPFh68OhUmuRPymKX9d 2019-03-27      in  viral  
2  5WHTFyqSii0lmT9R21abT8 2019-03-27      in  viral  
3  0A5gdlrpAuQqZ2iFgnqBFW 2019-03-27      in  viral  
4  7COXchtUOMd6uIT6HvmRaI 2019-03-27      in  viral

In [20]:
#are their songs that are in the charts of muliple countries ?
pd.pivot_table(viral_df_basic_top20,index=["Track Name",'Artist'], values=['country','ID'], aggfunc={'country':np.sum,'ID':'count'}).sort_values('ID',ascending=False).head(5)

ID  \
Track Name                           Artist            
Dance Monkey                         Tones And I  89   
death bed (coffee for your head)     Powfu        61   
Roses - Imanbek Remix                SAINt JHN    50   
ily (i love you baby) (feat. Emilee) Surf Mesa    50   
Sweet but Psycho                     Ava Max      50   

                                                                                            country  
Track Name                           Artist                                                          
Dance Monkey                         Tones And I  deesdemxnzmxbresidnzegmamaesegbrbrmxidtraregnz...  
death bed (coffee for your head)     Powfu        jpitzazaitarisusaugbcausgbruaritjpzacaauusisgb...  
Roses - Imanbek Remix                SAINt JHN    itzacaitarisusaugbcaisaucausgbruitgbcaaunzisgb...  
ily (i love you baby) (feat. Emilee) Surf Mesa    itzacaauitisarusgbisusgbgbusjpzacaauauuaitcais...  
Sweet but Psycho                     Ava Max      zaarititaritzaauususgbaugbzadejpjpgbnzauismaca...

In [21]:
#anything empty?
viral_df_basic_top20.isnull().sum()

Position       0
Track Name    12
Artist        13
URL            0
filename       0
ID             0
date           0
country        0
group          0
dtype: int64

In [22]:
#NAN values
viral_df_basic_top20[viral_df_basic_top20['Artist'].isnull()]

Position Track Name Artist  \
2700          1        NaN    NaN   
3652          3        NaN    NaN   
7113         14        NaN    NaN   
8558          9        NaN    NaN   
8564         15        NaN    NaN   
8566         17        NaN    NaN   
12561        12        NaN    NaN   
15316        17        NaN    NaN   
15509        10    NO GOOD    NaN   
19060        11        NaN    NaN   
20508         9        NaN    NaN   
20861        12        NaN    NaN   
20866        17        NaN    NaN   

                                                     URL  \
2700   https://open.spotify.com/track/47rbjDud83d3aqv...   
3652   https://open.spotify.com/track/74kCf4Yd2jr91yi...   
7113   https://open.spotify.com/track/4LAgGxPsaI4HeOJ...   
8558   https://open.spotify.com/track/2IHTNejiUvoSTsF...   
8564   https://open.spotify.com/track/4xioHVjM8WJmLEV...   
8566   https://open.spotify.com/track/6eiVcW005z9GU9D...   
12561  https://open.spotify.com/track/2AgPxySIeAnXDHZ...   
15316  https://open.spotify.com/track/47rbjDud83d3aqv...   
15509  https://open.spotify.com/track/4Qnz8tARYhUtDNe...   
19060  https://open.spotify.com/track/461JKAn7H6Sbx0q...   
20508  https://open.spotify.com/track/60pysSgEslc7i5b...   
20861  https://open.spotify.com/track/3x5RCKLhqj58oye...   
20866  https://open.spotify.com/track/5RKClHNCc5x2Bri...   

                      filename                      ID       date country  \
2700   2019-04-26-in-viral.csv  47rbjDud83d3aqvbcTssei 2019-04-26      in   
3652   2021-01-01-br-viral.csv  74kCf4Yd2jr91yinngEt0P 2021-01-01      br   
7113   2019-11-22-mx-viral.csv  4LAgGxPsaI4HeOJStF8tgV 2019-11-22      mx   
8558   2020-10-17-mx-viral.csv  2IHTNejiUvoSTsFpovaESS 2020-10-17      mx   
8564   2020-10-17-mx-viral.csv  4xioHVjM8WJmLEVd5DD8BQ 2020-10-17      mx   
8566   2020-10-17-mx-viral.csv  6eiVcW005z9GU9DpoEdlqh 2020-10-17      mx   
12561  2019-06-25-es-viral.csv  2AgPxySIeAnXDHZvAQHg0R 2019-06-25      es   
15316  2019-05-26-in-viral.csv  47rbjDud83d3aqvbcTssei 2019-05-26      in   
15509  2020-07-19-jp-viral.csv  4Qnz8tARYhUtDNePNsLx2C 2020-07-19      jp   
19060  2020-12-16-in-viral.csv  461JKAn7H6Sbx0ql9IvRUG 2020-12-16      in   
20508  2020-05-20-br-viral.csv  60pysSgEslc7i5blU5zZbS 2020-05-20      br   
20861  2019-12-22-mx-viral.csv  3x5RCKLhqj58oyeZMaNOmF 2019-12-22      mx   
20866  2019-12-22-mx-viral.csv  5RKClHNCc5x2BritkOF7tI 2019-12-22      mx   

       group  
2700   viral  
3652   viral  
7113   viral  
8558   viral  
8564   viral  
8566   viral  
12561  viral  
15316  viral  
15509  viral  
19060  viral  
20508  viral  
20861  viral  
20866  viral

In [23]:
# how many songs per country 
viral_df_basic_top20['country'].value_counts()


it    760
nz    760
au    760
tr    760
gb    760
us    760
id    760
jp    760
es    760
ca    760
mx    760
br    740
ar    720
de    700
za    700
is    680
eg    540
ma    540
in    440
ru    140
ua    140
Name: country, dtype: int64

# CALL API FOR MUSIC FEATURES

### FUNCTION: GET MUSIC FEATURES FROM API

In [26]:
def getTrackFeatures(df):
    feature_names=['ID','duration_ms', 'popularity', 'acousticness','danceability', 'energy', 'key','instrumentalness', 'liveness', 'loudness', 'mode','speechiness', 'valence', 'tempo','loud_start']
    features_df=pd.DataFrame(columns=feature_names)
    
    causing_error=[]
    
    for id in set(df['ID']):#only loop through the unique IDs to enhance performance
        try:
            print(id)
            meta = sp.track(id)
            features = sp.audio_features(id)
            analysis= sp.audio_analysis(id) #does not seem to be available at all songs

            #collect in dictionary
            features_dict = {}   

        
            # meta
            features_dict['ID']=meta['id']
            features_dict['duration_ms'] = meta['duration_ms']
            features_dict['popularity'] = meta['popularity']
            # features
            features_dict['acousticness'] = features[0]['acousticness']
            features_dict['danceability'] = features[0]['danceability']
            features_dict['energy'] = features[0]['energy']
            features_dict['key'] = features[0]['key']
            features_dict['instrumentalness'] = features[0]['instrumentalness']
            features_dict['liveness'] = features[0]['liveness']
            features_dict['loudness'] = features[0]['loudness']
            features_dict['mode'] = features[0]['mode']
            features_dict['speechiness'] = features[0]['speechiness']
            features_dict['valence']=features[0]['valence']
            features_dict['tempo'] = features[0]['tempo']
            #analysis
            features_dict['loud_start']=analysis['sections'][0]['loudness']#loundness at the start
        
        #error handling
        except:
            causing_error.append(id)
            print('causing error: ',causing_error)
            
        #wrap up of all gathered features
        track_df = pd.DataFrame(features_dict,index=[0],columns=features_dict.keys())
        features_df = pd.concat([features_df, track_df], axis=0, ignore_index = True)#concatinating along axis 0
    return features_df
    #lst_columns=['Position','Track Name','Artist','URL','filename','ID','date','country','group','duration_ms', 'popularity', 'acousticness','danceability', 'energy', 'instrumentalness', 'liveness', 'loudness','mode', 'speechiness', 'valence', 'tempo','loud_start']
    #df_with_features=pd.concat([df,features_df],axis=1,ignore_index = True)
    #df_with_features.columns=[list(df.columns)+list(features_df.columns)]#creates new joined list of columns
    #return df_with_features

#### call API on IDs returning a dataframe with features of each ID

In [27]:
#dataframe of features
viral_features_df=getTrackFeatures(viral_df_basic_top20) #get features for each song


3g9yjJFOtZkGAYoVz0WpeJ
5hcWlcZE9vFMeaFaTjo1Zf
6kPJZM97LwdG9QIsT7khp6
2z0ZSojJWKDOCucLfTNU3K
3v6a43VROJNCWoJXTCprLu
2kJwzbxV2ppxnQoYw4GLBZ
2TRGNVokPnC2a2oHg5iv1H
2Sla5cgNpVoRcckYjaAMKP
62YbYwUyKWBIjxrgobDWML
6HtVFR6WAeJRgYtleNcEE8
1xT34fo4r7js5GIDpybGgb
1IBUPdwV9TvaipSODdt8QY
4yOT0bYT83QZ0aXlJ3Tb2p
0OMFMPdrmtzKGrzlsfJpsS
0GVKkD74xcixRYYG3zv1NL
5jZFFTqz6ALemEVcdlcVbZ
6eK68flOp1XqQtbWohsdNE
29qhmx5mWhqHHXDCr1RDJV
1zFfc0vCtYNhH640Xgv6M3
5ollDCofTmRZN0Bbhz4EDW
55zZbsGOfKewealiRYeyAx
76zoaKNB1J4eOAWInCod2C
1quuqraz6z6RusyJZtyDcm
2IBoAGhwtoBwp0RD2kb9Kb
2kQTQ4x3tG1BcBA4nFp6qf
72vfLs4pNjEvfOWsp9cva3
3u1S1OmAUhx5DRlLrXqyp3
7zmleW3XZx0uUsL2CkFuDe
3QqIm5mpKvYMn4fJpAkObd
0AdRVtUvbbuUBOpcBEOIat
1dytqMU5Q8rgs5AmaW3n2z
4PlXXrdWJPqjerdLWSJk0b
3jkmf9ADm3wn9wRwZY0mDE
31EKU3oevb6I4up2oSYyfS
2an1crKGNWXdviPTzWyH0I
23H7ZCWvlS1SUNo3PWsRC5
18dts4OqWOQTGFlMTGodkk
6btUkPMi2hC2rfgCBIuhK3
0hPJnLTGMk9Wzl5tWDNBmB
1ThmUihH9dF8EV08ku5AXN
52oeIc4mgVs9J0o0QuZEag
5bQ1qgqZhpCS9RQCUZaqBO
17cgSaA0T3xBGzLQYLv20f
5vnuJ8EopkP

3pLUJWBRx9PBdphuNxn3Ka
71swOnXQAcrNT4O1xAlMUd
6qVA1MqDrDKfk9144bhoKp
2MX5LM5RFnYoFPRO3mehfG
2rPE9A1vEgShuZxxzR2tZH
7tLBfOAgC2j3UWqeGgZ7G2
3LS0vrBpJx8lhKAp7GhojE
66YgptYp9124vxw9yFsIGn
0C0ESa1ZZ5taXaYUxwnvx9
2IP8GvokmUkJ0HIopbZVsv
21pk7rsFoQESMSuBi8FVYr
0XPsOSYzDJZJArevQNm2AR
79s5XnCN4TJKTVMSmOx8Ep
7g8YaUQABMal0zWe7a2ijz
7IXi8qEDc9oASO5LTx19nT
4OxyYXWVqfqkLIwzffmbLS
1gTQBwtXBDwBQU77S02MeK
0xUvi7wldroBzaZCengPa5
3JN6cbMhHVyI2UbABfKVhY
3OzheOqhU7mOPYJhUMza27
6L3VsjOK656g3MEqckfVzA
7awUp2mRP7xjX9wBKhuwRu
5FCoiKyUuwEqAZxvvRio4p
0b0zNU1Ws6XZzFhk8cWI5J
2zqxUV6h5y6Y9WhyH0FVvX
0jT8Nl0shPS8115is0wD2Q
0YU17F0BlVXvmx5ytsR43w
75ZLfnMkHpmSQD2QyzBOtH
3UTUi5TYYKSTSer95YhiAv
17VsiwLi1ovKYdVUqDieGC
4D0GT8zWuA7Z2em6dpajAn
5UgZoWvuycExNhZd6vIF7D
7bfT9te1VZlMY5mO5ESi8H
0Rm9k5ZmONBvicI8gohVSc
63FbZONuRQLexBqNhedcMD
5hNlXT6ICe300iRWBc2PYp
2J9B63FawlTaPdg4eH5X03
4oTUA7Xj0ysY8fpJdGGHkT
2maGaB3fNqQnyrG3FDiaW3
5TVpVYcU47B23onx6llnvv
461JKAn7H6Sbx0ql9IvRUG
1gDEXpspQ9pPluwFsRFigl
0BjEQJZy4rfUNpKHiaYirL
4y3OI86AEP6

1bMQEcPrtNOLo8I5km7bzF
7lYXrmbJhunVGUYDGvZcL5
1G3UtySE4dLlDaDNv6rXyh
3zl7j5ua8mF4JDYuxrfo01
0ije1VnIvXnntrIiLt3FRI
6eiVcW005z9GU9DpoEdlqh
5WmPL0RQxooB4si6PuSkUi
3E1FgPTXX203rgBTD5nlKF
32h6yQGn4oAhZ0HRsTMX26
3Ghyp3X5Ua9ypHnBIgOObI
0Q6AD7e7d9tpN1dXL4qh5P
0ZW94xTUKrsNDXAYMsMXEv
5mUXM7NLShbIUXxP80lQjy
0YEIUeEHbloZzlk3Z6Wvuf
5Y4IMG8dZRnshdtVVaGNvA
7lVVjCI15ZUMbcDjUNoB4m
1e2ofLpE40cjEeMENP7pzv
5rsylaCMgQ5q5IjrZBGm8q
7fT5flhPP1Hy5kYyO98SU0
1lZiBN2IGHihxqf0vzqtjP
1KFN3kB4hc6r7ILpifFsqf
3jtCPldxNkWPPYyVH7kQyp
2vuFdbSOLje3khV4rLSEpm
0hEcycOJb4hjUPLKQEx44E
1U3tv41tFRgOfbytpuYQrR
00iRsSA9GeWcSeUtqpad84
0TJpnfkl1YrpbUUWtEyKAt
2RbVD1IgXtyQFlLtbthaZ3
76q0wuIpT0wIqiv5M21jEg
4pN7yDeBmmAud5lBtuCsVs
3V9cM3nCH2G66afoDi0snu
0qc4QlcCxVTGyShurEv1UU
7FGq80cy8juXBCD2nrqdWU
3NYdvUDBa2Y5UNhqH4INC1
5tZWtGT65sMrDVPB5H444n
6Za3190Sbw39BBC77WSS1C
65vseJi4kug0grGTTSmH2u
3VvA1wSxukMLsvXoXtlwWx
675w3ACw5xZR3ODhUJlEVN
05ysoVhSCvSj6GAEfNUtHF
03Pg5wDlaYAZlTNLeAfl2r
1WO8bnGxH56re21qefEcAt
4KhTyJ1cJeHaY4ZS2u3e6A
3rp5EbCFusp

678cJe437rxCAUzgqse4vg
2IBZ2ROgyUewCGCgWAp1sq
4zWO4gvuFtw6EJZC5FFGlr
0R5CKHJnL81mlddsCqH8IB
2DQNYMRjisN3KAPj1bEwOI
5GBbUx3eAHDgoOvdcpFObI
7teD59fnvnArwz3CIhORNT
5SBFZEipvLrddLiaDrZp2c
6FRoRSdJqU7wFb8XW7B0oR
2ksOAxtIxY8yElEWw8RhgK
2qlTyN8Ou086f0HTtZtGoA
2lYTJK94hb0fd1LQtb6Dhk
1fLJxyaInJxS2Yo6PPGxZq
4nV21C0UjO3rNrv5mvm69C
6Ib0IA41d0ksbP6o8oT1uk
66qYWzjiH7jlPfpoU3IxvV
1ue420NRDPZl5OQ4e6tyoA
1H4pOx4uI7BdxM2RL3D99f
1FMNuRMIcPD6PDQGoi5Ifn
4VRE2Ler8le7DUymvKepY1
7Dz0FxiogLMDI23GJ3FajC
7gAyu0od48mkaCw1laPKVC
2gZ703Lv0tNN40iKt7wbXG
0lsRatBUs9HNIZAmoGABzk
5MPKeueerGrcoUD8qPsAmX
5YmeqbDcLOEYwRJvctsWYj
4ZNqJS2x2bly4i258hK6E9
6YIkR7cwUp21KPsoFunDxf
1FTCYQMKDpsxoEVmHBZupb
0jtTDKt1FRTnKbWtWSGLoy
4SWu9u2dLzmYa8dMieyayQ
4sCabvmZpbmfoCNcoCSKEX
6s2xT3MAviRFKr0s7GSD2Z
47KSukHvyu6vg5giBWeN2g
4wyNKAZe82BNlp2lHiR1FX
0mfEHC4z5a7533o7VNOH2c
2Z4VK9otSeq3MxWcEgptUa
4GUj9MfPPkjpZskxb45uiX
2FDP440DvFJ0zbuCAqa6Ew
3EjZiWj3iZJMSoiSA8a89j
1WM0IXvQ23Gix7yVH4c2Du
2lO3VrLE2NteGoGjlDDnFA
0kzGjSRHGVcaWxagzQwAyD
0gD6lWJhbXH

4Eefey3uN90Abhbr47zKmC
4nSLNgCnCm0NX1SkOwrrdR
6dwwYV6O8VL0aWDz2GcK9U
70avaeZ57OiVydfPG1VyPL
2HGymMoOdB4ZYv0AAqYmgF
1AqemxWsT0iGg5Lvkm16Bt
233lqmrvWyIC7u5GiAnTV4
699vChUaGvvlGuwpItAIVY
0U9zlnKpuMqCi0GBUEvsQk
4poybupy26pTWPr9zY1i7h
07dRjnkeSazozZ16S07Nee
6u24qPPY0w9rJV8WnF6cpn
6t94PGXYAuo9GbJEnMpPcW
04ZTP5KsCypmtCmQg5tH9R
0QUgSlWpeW7VCkLmrc58RU
0QPlafvsnmTJBvGNek2cI8
7fGt7C5fZRgGsgRLNfeS0h
1O74aTse7W5hd2rAvnEVaW
0EH9z7yRcMxYebvQXKLGMr
0Xne1P32xl9Blq5GT0GqWU
24hPnL2Yr82INEk0YUWCJr
37hCsK2djczyRGWMjqzbxD
1kMUGCt7XLDc5Q64Vl0H8S
7Gq2ssuEmAx7oVy8frLLdN
4fndbjoz1qJyK6JcLdKfzm
6UKWtT7Ux09NHTEpcKTXX4
3xBYCXi1LBsQPGE08M3iAP
1UYY7AvG6Q5tFjFBQ40wrf
1GGBmiyTHOMBgvIlYqIBuq
5Xhqh4lwJPtMUTsdBztN1a
2ZGgIXaoierTsnkrDzuBzY
7iHMGmAOvGT29H4glknJum
7KXjTSCq5nL1LoYtL7XAwS
6KZWgj9osRBSQdukDSQKcL
2RrZbxdl9nuFMCrO71tXTG
3xGD5KhnizETt7KQwxtUAc
1tNJrcVe6gwLEiZCtprs1u
3bdjxhq0PqcxNwswCOX6w1
1GOiMxQXZUzjzH0Sxpck0B
5UHt8da1DadBV8Gsgtjg6t
2UM9VMeAnwvO0RSM3V5Gpn
1fHfF8eW8uTM9JlP0vPdPR
0bdYJCO9clK9ew61Z7GqMf
0tAr21qoXD1

...

2dpaYNEQHiRxtZbfNsse99
7hcXVytGpjqpw4Z64TylYn
5vGLcdRuSbUhD8ScwsGSdA
1c44jkn3UtBDXoU24LABa7
0DRw3l8ruUsRAXMJd7HITq
7G8jc5eGFmEWctZq98ONB6
6b2RcmUt1g9N9mQ3CbjX2Y
1Hhp6UDB1xjrbl5dOEqUWJ
19op3QHIz9Aq5VdwsASXIz
3dLiBBKbeUIhzGNeuo5JGS
7754vHc34U2wQMIJFWRhVK
5Zsy1AH2ZusZNUzn0CU6I5
21pObgkjeIl4QYHLdg8yeo
06agGBuAaolKYrIwPyvSVU
1WXfRSlXj23JjZEL1BP1nv
47iBtK4SMDXM2i2evdEvKx
78shOpmTfZKkkePtBbhwaU
1E7RHPXA9sZ30qfms18J7J
5FIOyIWVSnq3YbesfPsEyD
50Td3qilgs8BLtv8mHyT1t
184XtKkzoAl83quIarUAJP
4u2hiASkRpjQdwBIZQS4iF
1v9OR23wz65Tkw4vaqo5t1
3fEpuoSr8ufvyRR7s2jB3U
0sIywArMywWkp2ub0J6RGn
1UOWNaUMhOxXxSZHGhVDVj
4fl2CBee3AX4LbMUyB3OQA
4EjUFjzi4ErWjklOA3Aslu
07RbHF9ZBqsxZDrTndn1Ww
3QWpTf2VIdELYUUiKJAXlI
3qLbEQPE0mktP3s6JHn5oM
0Y4hwUppbGc8aIBmiTz30A
2kUT8dTjyum9wGfikCBmMA
3VPOZWtj8mM89IiRNygxgS
7nuqChXN3JyzKgkxmS9Rzc
5EWlefzWuFXfuQinl0jenC
4ykSWyBpdEhgVyTMVoLtoO
641R77CBy4bOLGRcA9sgHy
2kS6td1yvmpNgZTt1q5pQq
66yik17dX3eiuS5PetLJ0X
4W3FJapn5CD6Fnrlfj4HFe
56XuQIth58NVAePH49fsmX
05rNdZ5riKWXsb7iqbarqV
0yx82WHSiVR

2lAyctg5FocS67eksHoyIg
1Wp1gcHOTQtRKzmgshTdXe
1JBqLoGfH1LnGTE0wzAatU
3aX2S19ORZgfGvEwtCAYjE
5I7cUbPlv4nVpusiWijV6y
3RomXYN1ndY86NNwFpV3mA
141xCZ6pvh4ymCsQkOFX80
5H0NnVnY1a7BguEmVTPQOR
6kOHug1s9Nd5gsHrX8irs2
28ltIq2Kff5ntb9Bi19RDZ
4WuKHgxdCJsaOBIh7yndty
1VJwtWR6z7SpZRwipI12be
253j40m8HGhSFUKOBADdTA
73Jaz0VyLFIZwRYOzHubiH
4dDMgEj5GpBTI1lvVlYPu7
0jGcmJ2AgzdKSrFIn1sViq
78C3YRa8QCgaJrFwVsqkA7
5YXBEgfxcKaMYvYi4wcFEy
18B52LHLrMFISSmLgaOdAF
4Uy3Q3h0VbWWyyiB1MmRoP
0Mo25vUf3YkhdZwZJ9go7b
20v9X8TV0AgwNGFT71OX3w
5j18Uhcf38hPQyUYKpfEWN
47maFtUKPECVkpKoQcFedD
4hqOmrCCJKyiuwDexd8c2t
3271XiQoOnVW3TzJhTvO8N
5wFzpBpNin4XIGIdEdEOdd
07NbBNizPciWZQu9ZmdWPB
0HFkCSXRTUHCYJ148PdD0v
1JbqLwB6HUPRu9zeZnKP5m
0OyfyAIHIdERPduy0omDj4
2NPfBHlrP9jb9aittQ62Uj
2B94bbI1a4LyBpK1439oP7
0Wv23nhntPTf4xvIcp6dgm
58jm1c4IKMFIFoj2Yi3WRB
3HMzCcVRL5fHNF0Uv73LFV
2XdgVioNvENCspUhMzvyjH
2z7wqrpcBQd3y061E8KypF
6Vit8tYLMWbAlqtwhaBzOc
3QIAwtEEDOrv0g5NKCGrXZ
5AKbyqDRAm2A7mDlSx5aBi
4D3RKT8L0njyGA4aq3KQoW
3mqOLGAouhuRUCSANgpC7n
2ahuyDHjbRh

2VVqxQhALecXvaArlZSVXr
4LZTRFPU4MDTzQuopPiW5K
48XfojJC87VKLlUotYV0Gm
26NEo4Ar57Ph1daWpZX4ke
20xxauHsSIf5CkDxcNW2DG
3oTB0j7PKUvKlhGgCPITbp
3gJEE75JLupZRzUd8iAQkZ
5mFLAVBmZaSksNNpgwvoeg
3aPUPt18WCBy8pYAWn77Py
7KRaGWguQmNRKHVROCInCS
1bhYympuYYM80b63IbCgQr
0fea68AdmYNygeTGI4RC18
3Uq45ipGutypFPmETfaoaH
70eFcWOvlMObDhURTqT4Fv
0tKl8ZVM6xJkhznvJAS3Nq
1QDzlqxjANVqrPdXjKBfhi
0AU2fc1n67EyrPXt6C70Lh
2noHL0PeH7WjRSLBjWxU4Q
1KfIZjptrvN6jUpUzzRYiD
6R3YNDKSDdXh4s8sKewqCD
6EzZn96uOc9JsVGNRpx06n
3Hl7sOdSj8rY4LsTShdpZ4
187Pge0WmJNoS73kaIORSE
0UqG6rPcMvTmSQh2iYGOdR
0Dfh9ZFqANpDSMnOLkdgze
0b9oOr2ZgvyQu88wzixux9
4EyBZN8AmXr8W1HR4vWumN
1UmtsXHRbWdd3GqlnHO7pm
45ZGEdLxCZvdMX9jWlsAAx
2UOyQB26wSQzM0jiTcUg50
67VXzMGtIrbj52tB6D4djH
4GMmJVKPN99XBuWGrW47e7
3qjHbrnu71IJ04InWjmjvJ
70lnL3QaSOIIyMa2X9aVRL
2X0XWf4WiyB3liZEqwtlXG
3wAEhWSx5GvfCkqJ1SPA5S
5h8VHxOBK1VWWj46T8fSly
1CBjYXLotHrkiykMbI7EaU
61eDLrQvquaTtZRnGFz9Kh
3K8Rb4f5LoROwzKTP7korB
3bBOTMuQiNNxvtXzk02CZY
3NxIbHwj2cimcm0O89XQZo
73wcFiFeCQurmHDQIcirbm
29ndXhUgrR5

6ezX8VMsXTxNi628n6O9Tj
3RKZzMSV2SAWCeuPYeY0yV
7FQpAFyAyYaQV3cVJ2Ce0x
5HRHVvNCQpKNno2IAo8esd
2yK8AHppPlBQCjbDk0WmUo
1xeLSOaNvDg68FGjIkJxkU
2MeEuwplwbbjZ7hxRI6viw
4qzZm5EIdFurBpDieEmVc9
0t1kP63rueHleOhQkYSXFY
00PesUdTmC3Y5xtZbZND0p
7oxSh7uiB74lWGVIv2Qtlm
6Fip1S0fTUcfYaeiehN84G
0JTjZnuciQH4M3620aMDtZ
1MtmXPxXP4ntSjfF5xnOyM
5kwgMFf58HjfRnn536rRUP
3gD4J3RUHy4OUuZ3qAiaiG
5FLSzKDtwoYn1ztlR6fDw0
01TdkJ0lorIzBsdks8X1Tb
58zsLZPvfflaiIbNWoA22O
47mTZdlkxSxwh1eljtYFlo
6mrKP2jyIQmM0rw6fQryjr
0dk1IMtJj58PMch8ycxfV9
4HXRJ3Bz49FEDeEOfdtUJO
5O0JIw4YvvTdKlN9l69JFQ
5E30LdtzQTGqRvNd7l6kG5
3aUTkuo4ObA6QPtZSu5NBq
2SU8KDnWeMNuMjsPvdCmiu
3OBBX1d1F1ygSZZOxLIkU8
2Lx3tIGzVSUca6lyPtnjBV
2Ih87kI2Z4eOKQOqwmjenh
5Ke9U0hKHJnXZQTb1Tk6HB
4Yxc55NX3tAXC2mHRAhtcW
0Ai6qtPfiEYGCqBGJYWD1W
4B2kFp1ulVz1FgwVYSShoj
7DcvwMAiqKJQD1rrdfxSDx
7M9w4C8WenlvWfNRAulZOX
4jTEbIwCMcHSwlSEiR4NlQ
5ZbNy41c4YvFBsemQnnkej
4dV3okcjbQwTcg4ObIGr4N
4lESS6vuruP6a79KWRaQou
2RMA3oUjMsBoh6rLaev6H4
2MhnrAz9dreS8zLzEuDflG
5JGM7K7wgP8Qc7Hov4hGH9
4hvVe7K3JyH

4XluehCQCnhhYD4cAd9Wfu
2wAiFWjRupWmnDkQcu91MF
6qQ6KlKg2z3UfTRySnV9fv
2Rb6nYmuLEblpq4KIAjngL
0OgGn1ofaj55l2PcihQQGV
4DGOPKH1BEKtjj6UiOFzp2
0HXPgjf1IIK4oXhbQEIrVd
7EhlWgNRxx5ywAekggTfwV
7ICtPoFGLbjHYosgKEdYdc
61GjfiruWKgpuD9oyMFcU6
6LSVZvLkol3hrioUAU2pNj
27OeeYzk6klgBh83TSvGMA
6iJsPWTBHzLcfGIz57KKLq
0LZkDuucd7c8eAaj9S4RJR
4cSSL3YafYjM3yjgFO1vJg
3gLiZAwPQXkw2GadZwfPjI
5WrGRCtRfqJF9tu8soYd3b
4vjHPBQZq20lxDI8qnV8bE
3tFXKG42xSRdOrf6TrDBF8
7FknBUKLWwjWszRK1N0jqe
317jDgApvz58HaEBRMUGBX
5vdp5UmvTsnMEMESIF2Ym7
317D6KWnGAFfoA5j2zZuNL
1sTwHeqQGrbnGJs9qj2XTW
5EXz4VLRsQIzpx4oW4oYSN
00zzSEe82JTtz68XlARnh7
1he4G9w0zh5GokInpOSINW
53pOZId3hTYgJWzdNZ6Iow
5pRGXCDY0eT6R1QicAn7Uk
56nsR8Td7CWVCQIGjznu4L
5uvUy2JmVczOxwzgfu4DLc
0MF5QHFzTUM2dYm6J7Vngt
11r8xMuVTS66RjtYgLneRw
6fMzgFFGzWCwiHMzDzNpp5
630sXRhIcfwr2e4RdNtjKN
5mNj2g5yg9vcC3MONjbgcD
1CmP16cGnlRGMcL0S7q50s
1d4m1qmn5Bu2Y7dXBCM1kw
7f9DsQEsIoeTwKxV6shsom
6aOs2sDgTYr478TFsEs8jD
1UDQlUffJtFPwKh138H7zS
5kQre7o8ky5DXeEPcyaCR1
6aNxF6lbcwr8iA8EHOW3Xr
6zZMfagNj5N

In [31]:
print(len(viral_features_df))

# null values left
print(viral_features_df.isnull().sum())
viral_features_df.head()

6008
ID                  0
duration_ms         0
popularity          0
acousticness        0
danceability        0
energy              0
key                 0
instrumentalness    0
liveness            0
loudness            0
mode                0
speechiness         0
valence             0
tempo               0
loud_start          0
dtype: int64


ID duration_ms popularity  acousticness  danceability  \
0  3g9yjJFOtZkGAYoVz0WpeJ      211408          0        0.2270         0.642   
1  5hcWlcZE9vFMeaFaTjo1Zf      203085         61        0.0354         0.682   
2  6kPJZM97LwdG9QIsT7khp6      222653         73        0.0441         0.737   
3  2z0ZSojJWKDOCucLfTNU3K      258765         40        0.0196         0.792   
4  3v6a43VROJNCWoJXTCprLu      242181         43        0.6720         0.763   

   energy key instrumentalness  liveness  loudness mode  speechiness  valence  \
0   0.470   2                0     0.248    -9.258    1       0.0310    0.607   
1   0.862   7                0     0.341    -4.824    1       0.1720    0.643   
2   0.636  11         6.66e-05     0.350    -4.546    0       0.0437    0.565   
3   0.665  10         5.08e-06     0.597    -5.676    0       0.1000    0.786   
4   0.423  11         2.95e-05     0.105   -11.023    0       0.0872    0.393   

     tempo  loud_start  
0  130.013     -12.184  
1  122.975     -10.973  
2  105.005      -6.356  
3   90.001      -6.641  
4  110.032     -18.863

# Leftjoin 
left join the 13282 charts entries with the dataframe of the unique 5872 music features

In [32]:
viral_df=viral_df_basic_top20.merge(viral_features_df, how='left',on='ID')


In [33]:
#drop rows with missing music data - most notably Baccara "Yes Sir I can boogey"
viral_df.dropna(axis=0, how='any', thresh=17, inplace=True)
len(viral_df)

13699

# EXPORT TO CSV
Analysis will happen in jupyter notebook 'analysis'

In [34]:
viral_df.rename(columns={'length':'duration_ms'})

Position                      Track Name                Artist  \
0             1                          Sucker        Jonas Brothers   
1             2                         Undrunk              FLETCHER   
2             3                Don't Call Me Up                 Mabel   
3             4                       Moonlight               Gaullin   
4             5                 i'm so tired...                  Lauv   
...         ...                             ...                   ...   
13696        16             ゴーカ!ごーかい!?ファントムシーフ!         ハロー、ハッピーワールド!   
13697        17                           Wager              ichikoro   
13698        18                     見ざる聞かざる言わざる  WEDNESDAY CAMPANELLA   
13699        19  インターネットじゃカッコいいのに feat. Chiyoko                   uno   
13700        20       Say Amen (Saturday Night)   Panic! At The Disco   

                                                     URL  \
0      https://open.spotify.com/track/4y3OI86AEP6PQoD...   
1      https://open.spotify.com/track/5SHhPFh68OhUmuR...   
2      https://open.spotify.com/track/5WHTFyqSii0lmT9...   
3      https://open.spotify.com/track/0A5gdlrpAuQqZ2i...   
4      https://open.spotify.com/track/7COXchtUOMd6uIT...   
...                                                  ...   
13696  https://open.spotify.com/track/6Mfr5EHIrMhZefe...   
13697  https://open.spotify.com/track/7kKEqFQaPTmQgW7...   
13698  https://open.spotify.com/track/3unHkbItvg3HXYm...   
13699  https://open.spotify.com/track/5KBshrM1lL0a9j1...   
13700  https://open.spotify.com/track/4qSsjDGXplb6422...   

                      filename                      ID       date country  \
0      2019-03-27-in-viral.csv  4y3OI86AEP6PQoDE6olYhO 2019-03-27      in   
1      2019-03-27-in-viral.csv  5SHhPFh68OhUmuRPymKX9d 2019-03-27      in   
2      2019-03-27-in-viral.csv  5WHTFyqSii0lmT9R21abT8 2019-03-27      in   
3      2019-03-27-in-viral.csv  0A5gdlrpAuQqZ2iFgnqBFW 2019-03-27      in   
4      2019-03-27-in-viral.csv  7COXchtUOMd6uIT6HvmRaI 2019-03-27      in   
...                        ...                     ...        ...     ...   
13696  2018-04-01-jp-viral.csv  6Mfr5EHIrMhZefeWuOPuiO 2018-04-01      jp   
13697  2018-04-01-jp-viral.csv  7kKEqFQaPTmQgW7wadHspJ 2018-04-01      jp   
13698  2018-04-01-jp-viral.csv  3unHkbItvg3HXYmIwZVwJW 2018-04-01      jp   
13699  2018-04-01-jp-viral.csv  5KBshrM1lL0a9j18Rx7k5S 2018-04-01      jp   
13700  2018-04-01-jp-viral.csv  4qSsjDGXplb6422X2INvFW 2018-04-01      jp   

       group duration_ms popularity  acousticness  danceability  energy key  \
0      viral      181040          9        0.0427         0.842   0.734   1   
1      viral      183428          0        0.4550         0.679   0.572   5   
2      viral      178480         79        0.2960         0.674   0.881   9   
3      viral      173066         72        0.0529         0.813   0.761  10   
4      viral      162582         11        0.1760         0.599   0.733  11   
...      ...         ...        ...           ...           ...     ...  ..   
13696  viral      218640         28        0.1580         0.575   0.960   8   
13697  viral      317811         23        0.0270         0.340   0.911  11   
13698  viral      232886          3        0.5640         0.689   0.594   6   
13699  viral      262010          3        0.0667         0.741   0.813   9   
13700  viral      189186         43        0.1670         0.491   0.867   6   

      instrumentalness  liveness  loudness mode  speechiness  valence  \
0                    0    0.1060    -5.065    0       0.0588    0.952   
1             2.41e-06    0.1270    -6.703    1       0.0781    0.542   
2             3.01e-06    0.0793    -2.853    1       0.1470    0.234   
3               0.0119    0.3370    -5.687    0       0.0850    0.290   
4                    0    0.2420    -7.058    1       0.2030    0.534   
...                ...       ...       ...  ...          ...      ...   
13696         0.000104  

In [35]:
viral_df.to_csv(r'data/analysis/viral_charts_songs_2017-2020.csv', index = False)

In [32]:
###########################################